In [1]:
# Import the research library 
import logging
import benchmarklib
from benchmarklib import BenchmarkDatabase, CompileType, GroverRunner, GroverConfig, calculate_grover_iterations

benchmarklib.setup_logging(logging.INFO)

from sat import ThreeSat, ThreeSatTrial

db = BenchmarkDatabase("3SAT.db", ThreeSat, ThreeSatTrial)

benchmarklib.quantum_trials - INFO - Database initialized: 3SAT.db (3SAT)


In [2]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_cloud", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")

In [3]:
# Configure Grover Benchmark
config = GroverConfig(shots=10**4, optimization_level=3)
runner = GroverRunner(db_manager=db, service=service, backend=backend, config=config)

benchmarklib.grover - INFO - GroverRunner initialized for 3SAT problems


In [4]:
# problem parameters
num_vars_range = range(9, 11)

In [ ]:
for compile_type in [CompileType.CLASSICAL_FUNCTION, CompileType.XAG]:
    runner.start_batch(compile_type)
    
    for num_vars in num_vars_range:
        # Accumulate circuits for this num_vars group
        for problem in db.find_problem_instances(size_filters={'num_vars': num_vars}, ...):
            optimal_grover_iters = calculate_grover_iterations(len(problem.solutions), 2**num_vars)
            for grover_iter in range(1, optimal_grover_iters + 1):
                runner.run_grover_benchmark(
                    problem_instance=problem,
                    compile_type=compile_type, 
                    grover_iterations=grover_iter,
                )
        
        # Submit all circuits for this num_vars as one job
        job_id = runner.submit_job()
        print(f"Submitted job {job_id} for {compile_type.value}, {num_vars} vars")
    
    runner.finish_batch()

benchmarklib.grover - INFO - Problem Instance ID: 1478
base_primitive._get_mode_service_backend:WARNING:2025-06-12 09:05:35,926: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
benchmarklib.grover - INFO - Submitted job d15d18d3grvg008j7gv0 to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 3852
benchmarklib.grover - INFO - Problem Instance ID: 1478
base_primitive._get_mode_service_backend:WARNING:2025-06-12 09:05:52,180: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
benchmarklib.grover - INFO - Submitted job d15d1c55z6q00087h5mg to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 3853
benchmarklib.grover - INFO - Problem Instance ID: 1868
base_primitive._get_mode_service_backend:WARNING:2025-06-12 09:06:12,923: A backend was passed in as the mode but a session context manager i

In [ ]:
"""
Simple Parallel Benchmark Driver

Strategy:
1. Sequential phase: Collect all work that needs to be done
2. Split work into chunks for each worker thread
3. Each thread processes its chunk independently with Batch context
"""

import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Tuple
from qiskit_ibm_runtime import Batch

def collect_needed_trials(
    db: BenchmarkDatabase, 
    num_vars_range: range
) -> List[Tuple[ProblemInstance, CompileType, int]]:
    """
    Sequential phase: Collect all trials that need to be created.
    No concurrency here = no race conditions.
    
    Returns:
        List of (problem_instance, compile_type, grover_iterations) tuples
    """
    needed_trials = []
    
    print("Collecting trials that need to be created...")
    
    for num_vars in num_vars_range:
        print(f"  Checking {num_vars} variables...")
        
        problems = db.find_problem_instances(size_filters={'num_vars': num_vars})
        
        for problem in problems:
            optimal_grover_iters = calculate_grover_iterations(
                len(problem.solutions), 2**num_vars
            )
            
            for compile_type in [CompileType.CLASSICAL_FUNCTION, CompileType.XAG]:
                for grover_iter in range(1, optimal_grover_iters + 1):
                    
                    # Check if trial already exists
                    existing_trials = db.find_trials(
                        instance_id=problem.instance_id,
                        compile_type=compile_type,
                        trial_params={"grover_iterations": grover_iter},
                        include_pending=True,
                        limit=1  # Just need to know if any exist
                    )
                    
                    if not existing_trials:
                        needed_trials.append((problem, compile_type, grover_iter))
    
    print(f"Found {len(needed_trials)} trials that need to be created")
    return needed_trials

def process_trial_chunk(
    trial_chunk: List[Tuple[ProblemInstance, CompileType, int]],
    worker_id: int,
    db_name: str,
    problem_class,
    trial_class,
    service,
    backend
) -> int:
    """
    Worker function: Process a chunk of trials in a single thread.
    Each worker gets its own database connection and Grover runner.
    
    Args:
        trial_chunk: List of trials to process
        worker_id: Worker identification for logging
        db_name: Database filename
        problem_class: Problem class for database
        trial_class: Trial class for database  
        service: IBM Quantum service
        backend: IBM Quantum backend
        
    Returns:
        Number of trials successfully created
    """
    if not trial_chunk:
        return 0
    
    print(f"Worker {worker_id}: Processing {len(trial_chunk)} trials")
    
    # Create thread-local database and runner instances
    local_db = BenchmarkDatabase(db_name, problem_class, trial_class)
    local_runner = GroverRunner(local_db, service, backend)
    
    successful_trials = 0
    
    # Use Batch context for efficiency
    with Batch(backend) as batch:
        for i, (problem, compile_type, grover_iterations) in enumerate(trial_chunk):
            try:
                print(f"Worker {worker_id}: Trial {i+1}/{len(trial_chunk)} - "
                      f"Problem {problem.instance_id}, {compile_type.value}, "
                      f"{grover_iterations} iterations")
                
                trial = local_runner.run_grover_benchmark(
                    problem_instance=problem,
                    compile_type=compile_type,
                    grover_iterations=grover_iterations,
                    skip_existing=False,  # We already checked in sequential phase
                    save_to_db=True
                )
                
                successful_trials += 1
                
            except Exception as e:
                print(f"Worker {worker_id}: Failed to create trial - {e}")
                continue
    
    print(f"Worker {worker_id}: Successfully created {successful_trials} trials")
    return successful_trials

def split_list(items: List, num_chunks: int) -> List[List]:
    """Split list into roughly equal chunks."""
    chunk_size = len(items) // num_chunks
    remainder = len(items) % num_chunks
    
    chunks = []
    start = 0
    
    for i in range(num_chunks):
        # Add one extra item to first 'remainder' chunks
        current_chunk_size = chunk_size + (1 if i < remainder else 0)
        end = start + current_chunk_size
        
        if start < len(items):  # Only create chunk if there are items left
            chunks.append(items[start:end])
        
        start = end
    
    return [chunk for chunk in chunks if chunk]  # Remove empty chunks

def simple_parallel_benchmark(
    db: BenchmarkDatabase,
    num_vars_range: range,
    num_workers: int = 4
):
    """
    Simple parallel benchmark driver.
    
    Args:
        db: Database instance (only used for initial problem collection)
        num_vars_range: Range of variable counts to benchmark
        num_workers: Number of parallel worker threads
    """
    
    print(f"Starting simple parallel benchmark with {num_workers} workers")
    
    # Phase 1: Sequential collection (no race conditions)
    needed_trials = collect_needed_trials(db, num_vars_range)
    
    if not needed_trials:
        print("No trials need to be created!")
        return
    
    # Phase 2: Split work across workers
    trial_chunks = split_list(needed_trials, num_workers)
    
    print(f"Split {len(needed_trials)} trials into {len(trial_chunks)} chunks:")
    for i, chunk in enumerate(trial_chunks):
        print(f"  Worker {i}: {len(chunk)} trials")
    
    # Phase 3: Execute in parallel
    print("Starting parallel execution...")
    
    total_created = 0
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all worker tasks
        future_to_worker = {
            executor.submit(
                process_trial_chunk,
                chunk,
                worker_id,
                db.db_name,          # Pass database filename
                db.problem_class,    # Pass class references
                db.trial_class,
                service,             # These need to be in scope
                backend
            ): worker_id
            for worker_id, chunk in enumerate(trial_chunks)
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_worker):
            worker_id = future_to_worker[future]
            try:
                trials_created = future.result()
                total_created += trials_created
                print(f"Worker {worker_id} completed successfully")
            except Exception as e:
                print(f"Worker {worker_id} failed: {e}")
    
    print(f"Parallel benchmark completed: {total_created} trials created total")

# Even simpler version using basic threading
def simple_threaded_benchmark(
    db: BenchmarkDatabase,
    num_vars_range: range,
    num_workers: int = 4
):
    """
    Alternative using basic threading (no ThreadPoolExecutor)
    """
    
    # Phase 1: Collect work
    needed_trials = collect_needed_trials(db, num_vars_range)
    
    if not needed_trials:
        print("No trials need to be created!")
        return
    
    # Phase 2: Split work
    trial_chunks = split_list(needed_trials, num_workers)
    
    # Phase 3: Create and start threads
    threads = []
    results = {}
    
    def worker_wrapper(worker_id, chunk):
        """Wrapper to capture results"""
        try:
            result = process_trial_chunk(
                chunk, worker_id, db.db_name, db.problem_class, 
                db.trial_class, service, backend
            )
            results[worker_id] = result
        except Exception as e:
            print(f"Worker {worker_id} failed: {e}")
            results[worker_id] = 0
    
    # Start all workers
    for worker_id, chunk in enumerate(trial_chunks):
        thread = threading.Thread(
            target=worker_wrapper,
            args=(worker_id, chunk),
            name=f"BenchmarkWorker-{worker_id}"
        )
        thread.start()
        threads.append(thread)
    
    # Wait for completion
    for thread in threads:
        thread.join()
    
    # Report results
    total_created = sum(results.values())
    print(f"All workers completed: {total_created} trials created total")
    
    return total_created

